# Testing with pytest

## Introduction

In this lab, you will learn how to write automated tests for your code. Tests check that your code does what it's supposed to do, usually testing known good inputs/outputs of your functions, but also corner cases and error conditions. Here's a simple example:

```python
def add_two(n):
    return n + 2

def test_add_two():
    assert add_two(2) == 4
```

The `assert` keyword makes sure that the test fails when the given condition is false. More details about that will follow below.

Even if you're not writing huge and complex applications, it pays off to start writing tests early. Spending additional effort on tests might seem like an unneeded burden at first, especially for very simple projects! But with a bit of practice, it'll be straightforward to write tests in parallel to your  new code. Writing tests pays off sooner as you might think, because you don't need to execute your code manually all the time to verify it does what you intended.

Tests allow you to introduce future changes to your code quickly, without having to be afraid of breaking things when changing your code. In addition, they provide additional hints about how your code works in different scenarios (or edge cases), as well as forcing you to look at your code from a different angle, often finding bugs in the process.

Python comes with an integrated [unittest](https://docs.python.org/3/library/unittest.html) module, but writing tests using it is unnecessarily complex. Thus, we will instead be using the [pytest](https://docs.pytest.org/) testing framework. A core maintainer of pytest (Florian Bruhin) is working at the INS, but this is far from a personal choice - in fact, according to the [JetBrains Python Developers Survey 2020](https://www.jetbrains.com/lp/python-developers-survey-2020/) with more than 28'000 respondents, pytest is by far the most popular testing framework:

![survey results](survey2020.png)

Unfortunately, the "Automate the Boring Stuff" book does not cover testing at all. Thus, the "Summary" in here is a bit more in-depth than in previous labs.

Additionally, here are some alternative resources:

- [pytest Tech-Webinar at INS](https://bruhin.software/ins-pytest/) (video part of this lab)
- [Effective Python Testing With Pytest – Real Python](https://realpython.com/pytest-python-testing/)
- [pytest: helps you write better programs — pytest documentation](https://docs.pytest.org/en/stable/)


## Here be dragons!

![Hic Sunt Dracones](https://upload.wikimedia.org/wikipedia/commons/c/cd/Lenox_Globe_Dragons.png)

Some of the topics we will cover in this lab are quite advanced. Please don't let that discourage you, and feel free to ask questions! It's a tricky topic, because testing can touch upon various more advanced topics you haven't learned about in detail yet. Nevertheless it's an important topic to learn about as early as possible, as can be a very useful tool for later labs you solve.

If this feels like too much, feel free to skip some of the later topics/exercises (e.g. about fixtures), but make sure you make yourself familiar with the basics of pytest.

## Installation

Before you can start using `pytest`, you will need to install it. To do so, please run:

In [ ]:
%pip install --user pytest ipytest

This installs both pytest itself, as well as [ipytest](https://github.com/chmp/ipytest) for integration in the Jupyter Lab.

Next, use the "Kernel -> Restart" menu to make sure `ipytest` is loaded, and finally run:

In [2]:
import ipytest

ipytest.autoconfig(addopts=["--color=yes"])

Then, to verify that pytest is called correctly, first run it as an external process via:

In [ ]:
!pytest --version

If this works as expected, make sure it's possible to run pytest as part of the notebook:

In [4]:
%%run_pytest[clean]
def test_nothing():
    pass

============================ test session starts =============================
[...]

tmpju7i_jp_.py .                                                       [100%]

============================= 1 passed in 0.01s ==============================


More about how this works below.

## Usage

### Outside of the notebook

In normal usage, pytest is an external command-line tool which reads Python code containing tests from a file.

In some exercises, we will use this more "regular" mode of operation rather than the more convenient notebook integration - either due to how certain pytest/ipytest internals work, or because we want to run `pytest` over the same code multiple times without having to copy-paste the code.

Typically, if your code resides in a `myapp.py`, you'd have a corresponding `test_myapp.py` file with your tests, and running `pytest` will discover them in the `test_*` file.

To do the same thing from the notebook, first, we use a special `%%writefile` command as the first line of a cell to write its contents to a file:

In [5]:
%%writefile test_via_file.py
def test_in_a_file():
    pass

Writing test_via_file.py


This kind of command is called a "cell magic" by Jupyter, because it does something special with the contents of a cell.

After running a cell with the `%%writefile` magic, you should see the file show up in the file tree on the left. Next, we can run `pytest` as an external command by using the special `!pytest` syntax:

In [6]:
!pytest test_via_file.py

============================= test session starts ==============================
[...]

test_via_file.py .                                                       [100%]

============================== 1 passed in 0.02s ===============================


In your day-to-day usage, instead of running `pytest` on the command-line, you can also use integrations in IDEs like VS Code or IntelliJ's PyCharm, which let you conveniently run individual tests right from the editor. It's still recommended to get yourself familiar with running pytest from the commandline, since various useful pytest features can only be used that way.

### In the notebook

With `ipytest` set up, we can use the special `%%run_pytest[clean]` cell magic, which saves the contents to a temporary file and then runs pytest over that file:

In [8]:
%%run_pytest[clean]
def test_math():
    assert 1 + 1 == 2

=============== test session starts ================
[...]

tmp058xzk4i.py .                             [100%]

================ 1 passed in 0.01s =================


Make sure you always use `%%run_pytest[clean]` rather than just `%%run_pytest`, to avoid collecting and running all previous tests you've written. It's currently not possible to set this as the default behavior, but an option to do so is currently [in discussion](https://github.com/chmp/ipytest/issues/57) with the `ipytest` maintainer.

## Summary

### Autodiscovery

When you run `pytest`, it will start in your current directory and **discover all files starting with `test_*.py`** (except when you pass filename(s) to it, as in `pytest test_things.py`). In the files it discovered, it will then find:

- **All functions with a name starting with `test_`**
- (All classes starting with `Test`)
- (All classes inheriting from `unittest.TestCase` for compatibility with Python's built-in test runner)

In this exercise, we will only cover plain test functions. In pytest, classes are only used to group related tests. You'll learn more about Python classes in lab 18.

### Assertions

To verify that a certain condition holds true, we use the `assert` statement built into Python. After the `assert`, there's a condition, the same way there would be after an `if` keyword. We can provoke a failing assertion even outside of pytest:

In [9]:
!python -c "assert 1 + 1 == 3"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
AssertionError


Python will only tell us that an `AssertionError` happened, but without any details (we run `python` as an external command here to get the unaltered Python output - Jupyter Lab and `ipytest` change the output in some ways).

This is where `pytest` comes in, which will interpret those assertions in a special way, so that it's able to give us more information. If we move the assertion into a test function and run `pytest` over it, we will see:

In [10]:
%%run_pytest[clean]
def test_wrong_math():
    assert 1 + 1 == 3

=============== test session starts ================
[...]

tmp6bzqpgfo.py F                             [100%]

===================== FAILURES =====================
_________________ test_wrong_math __________________

    def test_wrong_math():
>       assert 1 + 1 == 3
E       assert (1 + 1) == 3

<ipython-input-10-67533e027c88>:2: AssertionError
============= short test summary info ==============
FAILED tmp6bzqpgfo.py::test_wrong_math - assert (...
================ 1 failed in 0.10s =================


Here, it might be immediately obvious what's wrong, but for more complex situations (imagine values coming from a server or user input), this "enriched" output can be very useful.

**Note:** There are no parentheses after `assert`, the syntax is `assert some_condition_here`.

We can add additional information to be shown by using a comma. This is often useful when debugging something in the tests, by using `assert False, "some information here"`:

In [11]:
%%run_pytest[clean]
import sys


def test_additional_info():
    assert False, f"We're on a {sys.platform} system"

=============== test session starts ================
[...]

tmplpn1dw2l.py F                             [100%]

===================== FAILURES =====================
_______________ test_additional_info _______________

    def test_additional_info():
>       assert False, f"We're on a {sys.platform} system"
E       AssertionError: We're on a linux system
E       assert False

<ipython-input-11-60a514e0d01f>:4: AssertionError
============= short test summary info ==============
FAILED tmplpn1dw2l.py::test_additional_info - Ass...
================ 1 failed in 0.02s =================


In summary, pytest takes care of:

- Automatically discovering test functions
- Running them all
- Displaying failures in an useful way

## Options

pytest takes various command-line options to customize its behavior. The most common ones are:

### Output

| Option | Behavior |
| -- |: -- |
| `-v` (`--verbose`) | **Verbose output, i.e. show test names** |
| `-s` (`--capture=off`) | **Disable output capturing** |
| `--setup-show` | **Show information about fixtures being used** |
| `--tb` | Control traceback generation (`auto`, `long`, `short`, `line`, `native`, `no`) |

### Information

| Option | Behavior |
| -- |: -- |
| `--markers` | **List all available markers** |
| `--fixtures` | **List all available fixtures** |
| `--help` | **Show all available options** |

### Test selection

| Option | Behavior |
| -- |: -- |
| `-k`, `-m` | **Filter based on name (*k*eyword) or marker** |
| `-x` (`--exitfirst`) | Exit instantly on first failure |
| `--lf` (`--last-failed`) | Only run tests which failed on last run |
| `--ff` (`--failed-first`) | Run last failed tests first, then run the rest |
| `--lw` (`--stepwise`) | Run until the first failure, then next time continue from there |

We will use the ones marked in bold in this lab, but the rest might be useful once you've written a handful of tests for a project. Run pytest with `--help` to see all available options.

You can use those options with the `ipytest` by adding them to the same line, e.g. `%%run_pytest[clean] -v`.

## Output capturing

By default, pytest will only show output for failing tests. This means we can use `print(...)` in tests to output some additional information. If the test passes, the output from the `print` is not displayed. If the test fails (e.g. by doing `assert False`), the output for that test will be shown:


In [12]:
%%run_pytest[clean]


def test_passing():
    print("I'm a passing test")


def test_failing():
    print("I'm a failing test")
    assert False

=============== test session starts ================
[...]

tmpgkmu65yp.py .F                            [100%]

===================== FAILURES =====================
___________________ test_failing ___________________

    def test_failing():
        print("I'm a failing test")
>       assert False
E       assert False

<ipython-input-12-089d53948b2d>:6: AssertionError
--------------- Captured stdout call ---------------
I'm a failing test
============= short test summary info ==============
FAILED tmpgkmu65yp.py::test_failing - assert False
=========== 1 failed, 1 passed in 0.03s ============


This behavior can be overridden using the `-s` (`--capture=off`) option. This will cause output from tests to show immediately. Note that this will mean that output from the tests and pytest will be mixed, consider adding `-v` to make it more readable:

In [13]:
%%run_pytest[clean] -s -v


def test_passing():
    print("I'm a passing test")


def test_failing():
    print("I'm a failing test")
    assert False

=============== test session starts ================
[...]

tmp081gp5f9.py::test_passing I'm a passing test
PASSED
tmp081gp5f9.py::test_failing I'm a failing test
FAILED

===================== FAILURES =====================
___________________ test_failing ___________________

    def test_failing():
        print("I'm a failing test")
>       assert False
E       assert False

<ipython-input-13-089d53948b2d>:6: AssertionError
============= short test summary info ==============
FAILED tmp081gp5f9.py::test_failing - assert False
=========== 1 failed, 1 passed in 0.03s ============


## Markers

pytest lets us "mark" tests to group related tests together, even across different files. Those marks use Python's decorator syntax. You don't need to know how decorators work in detail to use marks, but if you're interested, there's a [Real Python guide](https://realpython.com/primer-on-python-decorators/) on the topic.

To mark a test, we need to do `import pytest` and add a `@pytest.mark.`*something* line (a decorator) right over the test function. For example, we could write:

In [21]:
%%writefile test_markers.py

import pytest
import time


@pytest.mark.slow
def test_slow():
    time.sleep(2)


def test_fast():
    pass  # do nothing

Writing test_markers.py


Next, we will need to register those markers in a `pytest.ini` config file, to ensure that pytest knows which markers exist and that we didn't accidentally introduce a typo in the `@pytest.mark` decorator. Currently, if we run pytest, we get a warning:

In [22]:
!pytest test_markers.py

============================= test session starts ==============================
[...]

test_markers.py ..                                                       [100%]

============================== 2 passed in 2.02s ===============================


To register the markers, we create a new `pytest.ini` file listing and documenting the available markers:

In [17]:
%%writefile pytest.ini
[pytest]
markers =
  slow: Tests which take a while to run

Writing pytest.ini


Finally, we can now run pytest normally - here, we'll include `-v` to see the test names being run:

In [23]:
!pytest -v test_markers.py

============================= test session starts ==============================
[...]

test_markers.py::test_slow PASSED                                        [ 50%]
test_markers.py::test_fast PASSED                                        [100%]

============================== 2 passed in 2.02s ===============================


If we now want to only run the slow test, we can do so via `-m slow`:

In [24]:
!pytest -v -m slow test_markers.py

============================= test session starts ==============================
[...]
collected 2 items / 1 deselected / 1 selected                                  

test_markers.py::test_slow PASSED                                        [100%]

======================= 1 passed, 1 deselected in 2.02s ========================


Note the **1 deselected** in the output above. Similarly, we can pass a Python-like expression to `-m`, using keywords like `and`, `or` and `not`. Note that we'll need to quote the argument to avoid it being split by the shell:

In [25]:
!pytest -v -m "not slow" test_markers.py

============================= test session starts ==============================
[...]
collected 2 items / 1 deselected / 1 selected                                  

test_markers.py::test_fast PASSED                                        [100%]

======================= 1 passed, 1 deselected in 0.01s ========================


Now we can see that the slow test wasn't being run, and thus the total runtime took almost 0s instead of around 2s.

To see all available markers (including pytest's built-in ones), we can run pytest with `--markers`:

In [26]:
!pytest --markers test_markers.py

@pytest.mark.slow: Tests which take a while to run

[...]

@pytest.mark.skip(reason=None): skip the given test function with an optional reason. Example: skip(reason="no way of currently testing this") skips the test.

@pytest.mark.skipif(condition, ..., *, reason=...): skip the given test function if any of the conditions evaluate to True. Example: skipif(sys.platform == 'win32') skips the test if we are on the win32 platform. See [`skipif` in the pytest reference docs](https://docs.pytest.org/en/stable/reference.html#pytest-mark-skipif) for more information.

@pytest.mark.xfail(condition, ..., *, reason=..., run=True, raises=None, strict=xfail_strict): mark the test function as an expected failure if any of the conditions evaluate to True. Optionally specify a reason for better reporting and run=False if you don't even want to execute the test function. If only specific exception(s) are expected, you can list them in raises, and if the test fails in other ways, it will be reported as

## Skipping

Sometimes, we want to skip a test in certain conditions - perhaps because a test can't run on a certain operating system. To do so, pytest provides two built-in marks, `skip` and `skipif`. Using the `skip` mark skips a test unconditionally:

In [27]:
%%run_pytest[clean]
import pytest


@pytest.mark.skip
def test_nothing_interesting():
    assert False

=============== test session starts ================
[...]

tmplrokre5e.py s                             [100%]

================ 1 skipped in 0.02s ================


We can add a skip reason, which is shown when `-v` is given (depending on the terminal width):

In [31]:
%%run_pytest[clean] -v
import pytest


@pytest.mark.skip(reason="Don't feel like running this")
def test_x():
    assert False

=============== test session starts ================
[...]

tmprdsu1ksg.py::test_x SKIPPED (Don't fe...) [100%]

================ 1 skipped in 0.02s ================


Using `skipif` instead of `skip` lets us add a condition as the first argument (giving a `reason` is then mandatory) - thus, the above is equivalent to:

In [32]:
%%run_pytest[clean] -v
import pytest


@pytest.mark.skipif(True, reason="Don't feel like running this")
def test_x():
    assert False

=============== test session starts ================
[...]

tmpz_sx1a9m.py::test_x SKIPPED (Don't fe...) [100%]

================ 1 skipped in 0.02s ================


Instead of the `True`, we can use any condition, similar like in an `if`.

## Parametrizing

Often, the same test should run with different sets of data. If we had a `power` function similar to what you've seen in the functions chapter:

In [33]:
def power(basis, exponent):
    result = basis ** exponent
    return result

We might want to test several different values to see how the function handles those. To do so, we could write:

In [34]:
%%run_pytest[clean] -v


def test_power_two():
    assert power(2, 10) == 1024


def test_power_ten():
    assert power(10, 3) == 1000


def test_power_zero_exp():
    assert power(10, 0) == 1

=============== test session starts ================
[...]

tmp7x5ijhfw.py::test_power_two PASSED        [ 33%]
tmp7x5ijhfw.py::test_power_ten PASSED        [ 66%]
tmp7x5ijhfw.py::test_power_zero_exp PASSED   [100%]

================ 3 passed in 0.03s =================


However, this gets cumbersome, especially when the test is multiple lines which would need to be copy-pasted.

Instead, pytest lets us add a special `parametrize` marker (note the spelling, "parametrize", not "paramet**e**rize" nor "parametri**s**e" - all of those are valid English spellings, but pytest only accepts the first).
In the decorator, we need to specify:

- The names of the arguments we'd like to parametrize, as a single string (*not* multiple strings!)
- Their values, as a list of tuples

Thus, we could rewrite the above as:

In [35]:
%%run_pytest[clean] -v
import pytest


@pytest.mark.parametrize(
    "base, exponent, result",
    [
        (2, 10, 1024),
        (10, 3, 1000),
        (10, 0, 1),
    ],
)
def test_power(base, exponent, result):
    assert power(base, exponent) == result

=============== test session starts ================
[...]

tmpmmdkmpo2.py::test_power[2-10-1024] PASSED [ 33%]
tmpmmdkmpo2.py::test_power[10-3-1000] PASSED [ 66%]
tmpmmdkmpo2.py::test_power[10-0-1] PASSED    [100%]

================ 3 passed in 0.03s =================


Note how pytest has automatically generated names for the different test cases and they still run as three separate tests.

## Fixtures

For more complex tests, often there's some kind of data or preparation needed by different tests. A very central concept in pytest to separate and organize such data or setup/cleanup needed for tests are *fixtures*.

To use fixtures, we define a *fixture function*, which returns a value we're later going to use in a test. A fixture function is a normal Python function decorated with `@pytest.fixture`. The function then typically does one of three things:

- Do some kind of preparation steps for a test (e.g. load a configuration file)
- Return some kind of data or object needed for the tests
- Return some kind of utility object useful for writing the tests

For example, we could use:

```python
import pytest

@pytest.fixture
def answer():
    return 42
```

to define a fixture called `answer`. Tests can now use this fixture by having an argument with the *same name* as the fixture function:

```python
def test_answer(answer):
    assert answer == 42
```

Note that the `answer` variable inside the test function is the value *returned by* the fixture function - in other words, imagine pytest doing something like `test_answer(answer())` when running your test.

If we now run all this, the test passes as expected:

In [36]:
%%run_pytest[clean]

import pytest


@pytest.fixture
def answer():
    return 42


def test_answer(answer):
    assert answer == 42

=============== test session starts ================
[...]

tmpm9k83ha_.py .                             [100%]

================ 1 passed in 0.02s =================


We can freely combine multiple fixtures to arrange our test setup as we see fit - tests can use multiple fixtures, and fixtures can use other fixtures themselves:

In [37]:
%%run_pytest[clean]

import pytest


@pytest.fixture
def institute():
    return "INS"


@pytest.fixture
def half():
    return 21


@pytest.fixture
def answer(half):
    return half * 2


def test_half(half):
    assert half == 21


def test_answer_and_institute(answer, institute):
    assert answer == 42
    assert institute == "INS"

=============== test session starts ================
[...]

tmphciliz6s.py ..                            [100%]

================ 2 passed in 0.02s =================


If we run pytest with `--fixtures`, we see all available fixtures, including built-in ones.

Fixtures also provide a lot of more advanced features, which we won't get into as part of this course:

- Fixtures can do *cleanup*/teardown, i.e. run some code *after* each test using them, for example to terminate an external process or close a connection to a database.
- pytest can be instructed to *cache* the fixture function result, so that the fixture only gets called once per test file (or even only once for the entire test session). This can be dangerous as it weakens the isolation between individual tests, but often is needed when a certain setup step takes a long time and would be too expensive to do with every test.
- Similarly to how tests can be parametrized, fixtures can be parametrized - for example, if we'd write some kind of tool which can talk to appliances from two different manifactures, we might want to run all tests against both of them and expect them to run in the same way for both.
- Fixtures can run implicitly (*autouse*), so that its side-effects (e.g. some kind of preparation) are done for every test, even if the test doesn't take the fixture as an argument.

## Built-in fixtures

pytest exposes much of its functionality using built-in fixtures. We will look at three of them in more detail: `tmp_path`, `monkeypatch` and `capsys`.

### tmp_path

With the `tmp_path` fixture, we can get an empty temporary directory for every test. It represents a [pathlib](https://docs.python.org/3/library/pathlib.html) object, details about which you will learn in the next lab. For this lab, you only need to know three things about `pathlib` objects:

- You can use the `/` operator to chain paths. If you have a `tmp_path`, doing `file_path = tmp_path / test.txt` will give you a new `pathlib` object representing a `test.txt` file in the `tmp_path` folder.
- You can use `.write_text(...)` to write text to a `pathlib` object. Doing `file_path.write_text("Hello World")` will result in a `test.txt` file containing `Hello World`.
- You can use `.read_text()` on a `pathlib` object to read the text in a file. Doing `file_path.read_text()` will return the `"Hello World"` string we wrote into the file earlier.

The `tmp_path` fixture is useful if we want to create a file, which is used in some way by our code under test - for example, an input file for a commandline-tool, or some data generated by our code. To keep tests isolated, pytest creates a new directory for every test:

In [38]:
%%run_pytest[clean] --tb=short


def test_one(tmp_path):
    assert False, str(tmp_path)


def test_two(tmp_path):
    assert False, str(tmp_path)

=============== test session starts ================
[...]

tmp2ozuz1jq.py FF                            [100%]

===================== FAILURES =====================
_____________________ test_one _____________________
<ipython-input-38-8f2a005ba3c4>:2: in test_one
    assert False, str(tmp_path)
E   AssertionError: /tmp/pytest-of-jovyan/pytest-0/test_one0
E   assert False
_____________________ test_two _____________________
<ipython-input-38-8f2a005ba3c4>:5: in test_two
    assert False, str(tmp_path)
E   AssertionError: /tmp/pytest-of-jovyan/pytest-0/test_two0
E   assert False
============= short test summary info ==============
FAILED tmp2ozuz1jq.py::test_one - AssertionError:...
FAILED tmp2ozuz1jq.py::test_two - AssertionError:...
================ 2 failed in 0.02s =================


Note how the two tests got different paths based on their names. If we write some data to a file:

In [39]:
%%run_pytest[clean]
def test_data_file(tmp_path):
    file_path = tmp_path / "test.txt"
    file_path.write_text("Hello World")
    assert file_path.read_text() == "Hallo Welt"  # this will fail

=============== test session starts ================
[...]

tmp9mias3q9.py F                             [100%]

===================== FAILURES =====================
__________________ test_data_file __________________

tmp_path = PosixPath('/tmp/pytest-of-jovyan/pytest-1/test_data_file0')

    def test_data_file(tmp_path):
        file_path = tmp_path / "test.txt"
        file_path.write_text("Hello World")
>       assert file_path.read_text() == "Hallo Welt"  # this will fail
E       AssertionError: assert 'Hello World' == 'Hallo Welt'
E         - Hallo Welt
E         + Hello World

<ipython-input-39-b3ad378ba581>:4: AssertionError
============= short test summary info ==============
FAILED tmp9mias3q9.py::test_data_file - Assertion...
================ 1 failed in 0.02s =================


We can see in the output above that the file is stored at `/tmp/pytest-of-jovyan/pytest-1/test_data_file0` and investigate manually, e.g. by printing the file via `cat`:

In [41]:
!cat /tmp/pytest-of-jovyan/pytest-1/test_data_file0/test.txt

Hello World

### Monkeypatch

The `monkeypatch` fixture lets us temporarily modify some state for a test. Usually we do this when our code calls some function which gets in our way for automatic testing, and we want to replace it by a fake function.

Consider this example using `pyinputplus`:

In [42]:
import pyinputplus as pyip


def format_mail_from_header():
    email = pyip.inputEmail(prompt="Type in your email address: ")
    return f"From: Luigi Vercotti <{email}>"

If we wanted to test it without splitting it into two different functions, the input prompt gets in our way for testing - when `pytest` runs the function, you don't want to type in an email every time!

We can solve this by writing a custom function which acts like `pyip.inputEmail`, but returns a fixed address:

In [43]:
def fake_input_mail(prompt):
    return "luigi.vercotti@example.org"

We can now tell pytest to replace the function temporarily while the test is running by using the `monkeypatch` fixture, and calling `monkeypatch.setattr(module, "function_name", new_function)`. A completed test would thus look like:

In [44]:
%%run_pytest[clean]
def test_format_mail_from_header(monkeypatch):
    monkeypatch.setattr(pyip, "inputEmail", fake_input_mail)
    assert (
        format_mail_from_header() == "From: Luigi Vercotti <luigi.vercotti@example.org>"
    )

=============== test session starts ================
[...]

tmpa6l3khzo.py .                             [100%]

================ 1 passed in 0.01s =================


### capsys

The last built-in fixture we'll look at is `capsys`. Recall how pytest *captures* the output done from tests, so that it doesn't appear when a test passes. Using `capsys` we can access this captured output, and e.g. test functions which use `print()`. To do so, the fixture provides a `readouterr()` function which returns a pair of the standard output (stdout) and error output (stderr):

In [45]:
%%run_pytest[clean]


def shout():
    print("We are the knights who say NI!")


def test_shout(capsys):
    shout()
    stdout, stderr = capsys.readouterr()
    assert stdout == "We are the knights who say NI!\n"

=============== test session starts ================
[...]

tmp2p3ajhuu.py .                             [100%]

================ 1 passed in 0.02s =================


## Exercises

### Exercise 1: Getting Started

Before you write your first "real" test, make yourself familiar again with how to run tests, both directly in the notebook and via an external file.

First, let's run a simple test (which does nothing) from the notebook directly:

In [ ]:
# todo: Adjust the cell to run the test, then run it.


def test_in_notebook():
    pass

Next, let's write the same test to a file and run it that way:

In [ ]:
# todo: Adjust the cell so that its contents are written to test_first.py, then run the cell.


def test_in_file():
    pass

In [ ]:
# todo: Run pytest as external process and pass the filename to it.

### Exercise 2: Your First Real Test

Below, you'll find a `censor_phone_numbers` function, using the regex from the last lab. Complete the function so that it replaces all phone numbers in the input string `inp`.

First, run the cell and then verify that it is working manually by calling the function and looking at the result.

In [ ]:
import re

PHONE_NUMBER_REGEX = re.compile(r'\+\d{9,15}')

def censor_phone_numbers(inp):
    # todo: Replace all phone numbers in 'inp' and return the result

print(censor_phone_numbers("You can reach the INS at +41552221838"))

Imagine we'd now want to improve the regex so that it also matches other phone number formats, for example containing spaces. Manual testing would get cumbersome quickly. Instead, let's write a first test for what we've just tested manually above.

In [ ]:
%%run_pytest[clean]

# todo: Write test for censor_phone_numbers with the same input as above

### Exercise 3: Output Capturing

We'll move away from our phone number tests for the next exercises, but we'll get back to them later.

In the summary above, we've mentioned how `print(...)` acts differently inside pytest, as pytest won't show the output for failing tests.

Write a test which fails, and one which passes. In both tests, use `print(...)` to show some output:

In [ ]:
%%run_pytest[clean]

def test_passing():
    # todo: Print some text

def test_failing():
    # todo: Print some text
    # todo: Fail the test (after printing!)

Run those tests. What's the difference? Run them again but pass `-sv` (`--capture=off --verbose`) to pytest. What changes?

In [ ]:
# Run the tests

### Exercise 4: Using markers

Next, we want to group related tests using markers. For this exercise, we'll write our code into a file, because ipytest has [an issue](https://github.com/chmp/ipytest/issues/39) which prevents it from showing certain pytest warnings.

To see how markers work, start with the following code:


In [ ]:
%%writefile test_markers.py

import time

# todo: Add slow markers


def test_slow_1():
    time.sleep(2)  # e.g. some expensive calculation


def test_slow_2():
    time.sleep(2)  # see above


def test_fast_1():
    pass


def test_fast_2():
    pass

Run `pytest` on the unmodified file and observe the runtime duration shown at the end of the pytest output. Next, change the file (edit the cell above and run it again) to add a `slow` marker on the slow tests. Then, register your marker:

In [ ]:
# todo: Write a file to register markers with pytest

Finally, run pytest again, but this time pass `-m "not slow"` as option, to only select the fast tests.

In [ ]:
# Run the tests



### Exercise 5: Skipping Tests

We're now back to running pytest with `%%run_pytest` instead of external files.

Imagine a test which can't run on your operating system. For example, on the Linux-based Jupyter Hub system, this test will fail:

In [ ]:
%%run_pytest[clean]

import os

# todo: Skip this test on Linux


def test_win():
    assert os.sep == "\\"

First, run the test to make sure it fails. Adjust the test to skip it if it's running on a Linux system (hint: see `test_additional_info` from the summary to get an idea how to do so). Finally, run the test again (try with `-v`) to make sure it's skipped.

### Exercise 6: Parametrizing

Let's go back to our phone number tests. In exercise 2 we wrote a single test for the functionality - but that probably doesn't cover everything we want to test. Some ideas:

- Does the phone number get censored if it's in the middle of a sentence?
- Does it work the same way if the string only contains the phone number, nothing else?
- What about a string which contains two phone numbers?
- Our regex checks for phone numbers between 9 and 15 digits. Do both of those extremes work as intended?
- If we pass a `+` followed by only 8 digits, or followed by 16 digits, does the number (probably not a phone number!) stay unchanged?
- If we have a 11-digit number, but with a letter in between, does that stay unchanged?
- If we have a phone number, but with spaces in between, does that stay unchanged?
- What happens if there's some text like `>>>+41...<<<`, i.e. some characters directly preceding/following the phone number?

Most of those questions you can probably answer by carefully looking at the regex - but tests serve as a helpful documentation about exactly those kinds of edge cases. If we change our regex to improve it, the tests make sure everything still works as expected.

Instead of writing many different test functions, we'll use pytest's parametrizing feature.

- First, start with your test from exercise 2, which tests a single value.
- Modify the test so that it does the exact same thing, but using the parametrize functionality (with the same single value for now, and the associated expected output)
- Expand the parametrization to cover all cases outlined above. Did you find something which might be a bug in our regex pattern?

Note we're running pytest with `-v` here, so that you can see the individual values being tested.

In [ ]:
%%run_pytest[clean] -v

# todo: Copy the test you wrote for exercise 2
# todo: Add parametrization

### Exercise 7: Fixtures

As a next step, we want to make our `censor_phone_numbers` function more generic: It should take a (compiled) regex pattern as an argument, instead of hardcoding the phone number regex.

In [ ]:
def censor_pattern(pattern, inp):
    # todo: Replace all texts matching 'pattern' in 'inp' and return the result

If we now want to write tests for the modified functions, those tests need to pass our pattern, `re.compile(r'\+\d{9,15}')`, to the `censor_pattern` function.

Right now, we only have a single test function (thanks to parametrization) and could easily use the regex pattern in there directly. However, once our tests and code get more complex, it maks sense to separate setup of objects/data needed for our tests into fixtures.

- Take your parametrized test from the previous exercise (or you could write a new one, without parametrization, if you prefer).
- Adjust the test function to use a new `pattern` argument (which isn't parametrized)
- Write a `pattern` fixture function for pytest which returns the compiled pattern

In [ ]:
%%run_pytest[clean]

import pytest


@pytest.fixture
def pattern():
    return


# todo: Add "pattern" fixture which returns pattern

# todo: Add test for censor_pattern

### Exercise 8: Using tmp_path

We'll now move away from our `censor_*` functions again, looking at a couple of built-in pytest fixtures. First, you'll learn how to use `tmp_path`.

- Write a new `data_path` fixture function, which uses the `tmp_path` fixture.
  * In that fixture, use `tmp_path` to create a new path object, pointing to a `data.txt` in the same directory
  * Then, write the text "Some data used for tests" to that file
  * Finally, return the path object pointing to `data.txt` from the fixture function
- Write a test, which:
  * Uses the `data_path` fixture
  * Reads the text the fixture has written into the file
  * Makes sure that the text is what we expect it to be
- Find the file in the file system using `!ls` and `!cat` (feel free to open a separate terminal tab if you find it more convenient). You should find it in `/tmp/pytest-of-jovyan` somewhere.

In [ ]:
%%run_pytest[clean]

# todo: data_path fixture

# todo: test using data path

In [ ]:
# todo: find file in the filesystem
!ls /tmp/pytest-of-jovyan

!ls /tmp/pytest-of-jovyan/pytest-2

!ls /tmp/pytest-of-jovyan/pytest-2/test_data_path0

!cat /tmp/pytest-of-jovyan/pytest-2/test_data_path0/data.txt

### Exercise 9: Using monkeypatch

The next fixture we'll look into is `monkeypatch`. We want to test a function using the `pyinputplus.inputMenu` function:

In [ ]:
import pyinputplus as pyip


def user_likes_python():
    response = pyip.inputMenu(["Python", "C++", "Java"])
    return response == "Python"

If we want to run a test for this function without any patching, pytest ask us for input while running the tests - try it out (and use the stop button to interrupt execution):

In [ ]:
%%run_pytest[clean]


def test_user_likes_python():
    assert (
        user_likes_python()
    )  # what should we even test here? The outcome depends on what we'd enter...

Instead, we now want to patch the `pyip.inputMenu` function away using `monkeypatch`, and make the test pass. To do so, we need to:

- Write a function to replace `pyip.inputMenu`, which always returns `"Python"`
- Use `monkeypatch` in the test, to replace the `"inputMenu"` attribute of the `pyip` module by our own function


In [ ]:
%%run_pytest[clean]

def fake_input_menu(...):  # todo: what arguments?
    # todo: return value

def test_user_likes_python():  # todo: add fixture
    # todo: use monkeypatch.setattr
    assert user_likes_python()

### Exercise 10: Using capsys

Finally, we'll combine the `monkeypatch` and `capsys` fixtures to test a function which prints something:

In [ ]:
import random


def print_interplanetary_greeting():
    planet = random.choice(
        ["Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"]
    )
    print(f"Hello to all potential lifeforms on {planet}!")


print_interplanetary_greeting()

If we want to test this code, we'll notice three problems:

- The output depends on randomness
- The output is written via `print`, not returned in a way we could easily test it
- Pluto is missing :(

To fix those problems:

- Write a function to replace `random.choice`, which always returns `"Pluto"`
- Write a test which does:
  * Use `monkeypatch` to replace `random.choice` with that function
  * Use `capsys` to access the output the test has printed
  * Assert that the output matches what you expect (make sure you include the final newline, `\n`)

In [ ]:
%%run_pytest[clean]

# todo: Write function to replace random.choice

def test_print_interplanetary_greeting(...):  # todo: add arguments
    # todo: Use monkeypatch to patch the "choice" attribute of the random module
    print_interplanetary_greeting()
    # todo: Use capsys to ensure the correct text has been printed

## Where to go from there

If you want to dig more into pytest, here are some further ideas for exercises (note: no solutions given, but feel free to ask your instructor for a live demo):

- Read the pytest documentation on [caching fixture values](https://docs.pytest.org/en/6.2.x/fixture.html#scope-sharing-fixtures-across-classes-modules-packages-or-session). Write a fixture which sleeps a couple of seconds (to simulate generating some data), play with the different `scope` settings and observe the result.
- Read the pytest documentation on [using fixtures implicitly via autouse](https://docs.pytest.org/en/6.2.x/fixture.html#autouse-fixtures-fixtures-you-don-t-have-to-request). Write multiple tests which need to use `monkeypatch` in the same way (e.g. to patch `random.choice` away). Move the patching into an `autouse=True` fixture so that it's done automatically for all tests.
- Find out how to write [lambda functions](https://realpython.com/python-lambda/) in Python. Revisit the test from exercise 9 above, and use a lambda in the `monkeypatch.setattr` call instead of the named `fake_input_menu` function. Next, parametrize the test with the value "chosen by the user" (i.e. returned from our fake function - either `Python`, `Java` or `C++`) as well as the expected result (`True` or `False`) so that our test can test all three possibilities.

## Wrapping up

After these exercises, you should now be familiar with how to write tests for your code! Feel free to ask questions if you got stuck somewhere, are confused about something, or just curious!

We encourage you to write tests for the code you write in the following labs, to get into a habit of testing your code automatically rather than manually. Some of the later labs will also introduce more pytest features we haven't covered here yet (e.g. asserting expected exceptions in tests).

While we've looked into a lot of different pytest features in this lab, testing is a very wide topic - we've barely scratched the surface! If you start writing more complex applications, consider looking through the [pytest documentation](https://docs.pytest.org/) for more advanced topics and best practices.